 # Opis notatnika
 Analiza przygotowana w poprzednim kroku została odebrana pozytywnie. W związku z tym, zostaliśmy poproszeni
 o przygotowanie raportu na podstawie danych z roku 2020. Ma być on później  wdrożony przez zespół IT na zewnętrzny
 serwis. Wybór padł na `Dash`.

 Zadanie wykonamy w dwóch krokach:  
 1. Znając wymagania raportu, stworzymy na bazie danych odpowiednie komponenty, które zostaną następnie wykorzystane do wizualizacji.  
 1. Stworzymy raport używając `Dash`.

 Ten notatnik skupia się na odpowiednim przygotowaniu bazy danych pod cele raportowe.
 
 W tej części projektu końcowego wcielasz się w rolę BI Engineera, który ma za zadanie stworzyć raport zgodny z wytycznymi biznesowymi dostarczonymi przez klienta.

 # Wymagania biznesowe
 Po prezentacji wyników analizy na niezależnym spotkaniu, zostały określone następujące obszary raportowania odsetka opóźnień lotów:  
 1. Wyświetlanie TOP 10 (w sensie najwyższego odsetka opóźnień) lotnisk na podstawie liczby odlotów, wyświetlona ma być również informacja o liczbie przylotów - widok `top_airports_by_departure`.  
 1. Wyświetlenie TOP 10 (w sensie najwyższego odsetka opóźnień) tras lotów. Przy czym istotna dla nas jest kolejność, przykładowo trasa (Warszawa, Paryż) jest inna niż (Paryż, Warszawa). Dodatkowym wymaganiem jest, aby minimalna liczba lotów odbytych na trasie wynosiła co najmniej 10000 przelotów - widok `top_reliability_roads`.  
 1. Porównanie roku 2019 vs. 2020, aby śledzić wpływ COVID na realizację lotów. Interesują nas podejścia:  
     - miesiąc do miesiąca, przykładowo odsetek opoźnień styczeń 2019 vs. odsetek opoźnień styczeń 2020, odsetek opoźnień luty 2019 vs. odsetek opoźnień luty 2020 itd. - widok `year_to_year_comparision`,  
     - dzień do dnia, przykładowo odsetek opoźnień wtorek 2019 vs. odsetek opoźnień wtorek 2020 - widok `day_to_day_comparision`.  
 1. Dzienny, czyli jak danego dnia, globalnie wyglądał wskaźnik opóźnień lotów samolotu, tj. odsetek opóźnień 01-01-2019, odsetek opóźnień 02-01-2019 itd.

 # Podejście techniczne do problemu
 Naszym celem będzie odseparowanie warstwy przygotowania danych (logika raportu) od warstwy prezentacyjnej (wizualizacja).
 Chcemy zapewnić, aby odpowiednie procesy zajmowały się tylko swoimi zadaniami. 
 
 > W tym podejściu warstwa prezentacyjna (wykres/raport) nie implementuje logiki biznesowej w celu przetwarzania danych. Innymi słowy, nie chcemy aby przykładowo agregacja była wykonywana w momencie tworzenia wizualizacji.

 To podejście będzie spójne ze współczesnym sposobem projektowania aplikacji. Ma to też dodatkowy benefit - ze względu na wolumen danych na bazie, nie musimy ich najpierw pobierać - warstwa logiczna je odpowiednio zagreguje i przekaże zdecydowanie mniejszą liczbę wierszy, co przyśpieszy działanie całości.

 > Logika działania tego notebooka jest zbieżna z tą, którą robiliśmy już na przykładzie `Inicjowania bazy danych` - warto mieć go pod ręką.

 # Przygotowanie bazy danych
 Na bazie danych, gdzie umieszczone są już dane, wszystko zawarte jest na schemacie `public`. Ponieważ zgodnie z wymaganiami otrzymujemy nowy obszar wykorzystania danych, stworzymy sobie schemat dedykowany - `reporting`.  
 Dalej stworzymy widoki, które odpowiedzą na zadane wcześniej pytania.

 > Stworzenie dedykowanego schematu ma więcej korzyści niż nam się wydaje, w ten sposób możemy łatwo wprowadzić zabezpieczenie na dane, które chcemy udostępniać.

 ## Stworzenie dedykowanego schematu
 W pliku `reporting.sql` napisz kwerendę, która stworzy (o ile już nie istnieje) schemat `reporting`.
 Ten temat nie był omawiany w trakcie trwania kursu, jednak łatwo można uzupełnić wiedzę czytając np. [ten](https://www.postgresqltutorial.com/postgresql-administration/postgresql-create-schema/) samouczek.

 # Aktualizacja bazy danych
 W tym miejscu odpowiednio skonfiguruj połączenie do bazy danych.

 Tutaj zaimportuj potrzebne biblioteki

In [96]:
import psycopg2
from psycopg2 import connect

 ## Konfiguracja połączenia
 Tutaj uzupełnij konfigurację połączenia

In [97]:
username = 'postgres'
password = 'SQLisFun'

host = 'localhost'
database = 'airlines'
port = None

 Tutaj zdefiniuj zmienną `con` oraz `cursor`

In [98]:
con = psycopg2.connect(user=username, password=password, host=host, database=database)
print("Połączenie udane.")
cursor = con.cursor()

Połączenie udane.


 ## Wczytanie pliku `reporting.sql`
 Z katalogu `sql` wczytaj plik `reporting.sql`

In [99]:
reporting = open(r'C:\Users\urszu\OneDrive\Pulpit\CodersLab - cwiczenia\Projekt koncowy\sql\reporting.sql')
reporting = str(reporting.read())
print(reporting)

/*
Tutaj zdefiniuj schemÄ™ `reporting`
*/

/*
Tutaj napisz definicjÄ™ widoku reporting.flight, ktĂłra:
- bÄ™dzie usuwaÄ‡ dane o lotach anulowanych `cancelled = 0`
- bÄ™dzie zawieraÄ‡ kolumnÄ™ `is_delayed`, zgodnie z wczeĹ›niejszÄ… definicjÄ… tj. `is_delayed = 1 if dep_delay_new > 0 else 0` (zaimplementowana w SQL)

WskazĂłwka:
- SQL - analiza danych > DzieĹ„ 4 Proceduralny SQL > WyraĹĽenia warunkowe
- SQL - analiza danych > Przygotowanie do zjazdu 2 > Widoki
*/
CREATE OR REPLACE VIEW reporting.flight as
SELECT 1
;
/*
Tutaj napisz definicjÄ™ widoku reporting.top_reliability_roads, ktĂłra bÄ™dzie zawieraĹ‚a nastÄ™pujÄ…ce kolumny:
- `origin_airport_id`,
- `origin_airport_name`,
- `dest_airport_id`,
- `dest_airport_name`,
- `year`,
- `cnt` - jako liczba wykonananych lotĂłw na danej trasie,
- `reliability` - jako odsetek opĂłĹşnieĹ„ na danej trasie,
- `nb` - numerowane od 1, 2, 3 wedĹ‚ug kolumny `reliability`. W przypadku takich samych wartoĹ›ci powino zwrĂłciÄ‡ 1, 2, 2, 3... 
PamiÄ™taj o t

 W tym miejscu odpowiednio rozdziel zawartość pliku `reporting.sql` na mniejsze kwerendy używając `;`

In [100]:
separated = reporting.split(';')

print(separated)

['/*\nTutaj zdefiniuj schemÄ™ `reporting`\n*/\n\n/*\nTutaj napisz definicjÄ™ widoku reporting.flight, ktĂłra:\n- bÄ™dzie usuwaÄ‡ dane o lotach anulowanych `cancelled = 0`\n- bÄ™dzie zawieraÄ‡ kolumnÄ™ `is_delayed`, zgodnie z wczeĹ›niejszÄ… definicjÄ… tj. `is_delayed = 1 if dep_delay_new > 0 else 0` (zaimplementowana w SQL)\n\nWskazĂłwka:\n- SQL - analiza danych > DzieĹ„ 4 Proceduralny SQL > WyraĹĽenia warunkowe\n- SQL - analiza danych > Przygotowanie do zjazdu 2 > Widoki\n*/\nCREATE OR REPLACE VIEW reporting.flight as\nSELECT 1\n', '\n/*\nTutaj napisz definicjÄ™ widoku reporting.top_reliability_roads, ktĂłra bÄ™dzie zawieraĹ‚a nastÄ™pujÄ…ce kolumny:\n- `origin_airport_id`,\n- `origin_airport_name`,\n- `dest_airport_id`,\n- `dest_airport_name`,\n- `year`,\n- `cnt` - jako liczba wykonananych lotĂłw na danej trasie,\n- `reliability` - jako odsetek opĂłĹşnieĹ„ na danej trasie,\n- `nb` - numerowane od 1, 2, 3 wedĹ‚ug kolumny `reliability`. W przypadku takich samych wartoĹ›ci powino zwrĂłciÄ

In [101]:
for i in separated:
    print(i)

/*
Tutaj zdefiniuj schemÄ™ `reporting`
*/

/*
Tutaj napisz definicjÄ™ widoku reporting.flight, ktĂłra:
- bÄ™dzie usuwaÄ‡ dane o lotach anulowanych `cancelled = 0`
- bÄ™dzie zawieraÄ‡ kolumnÄ™ `is_delayed`, zgodnie z wczeĹ›niejszÄ… definicjÄ… tj. `is_delayed = 1 if dep_delay_new > 0 else 0` (zaimplementowana w SQL)

WskazĂłwka:
- SQL - analiza danych > DzieĹ„ 4 Proceduralny SQL > WyraĹĽenia warunkowe
- SQL - analiza danych > Przygotowanie do zjazdu 2 > Widoki
*/
CREATE OR REPLACE VIEW reporting.flight as
SELECT 1


/*
Tutaj napisz definicjÄ™ widoku reporting.top_reliability_roads, ktĂłra bÄ™dzie zawieraĹ‚a nastÄ™pujÄ…ce kolumny:
- `origin_airport_id`,
- `origin_airport_name`,
- `dest_airport_id`,
- `dest_airport_name`,
- `year`,
- `cnt` - jako liczba wykonananych lotĂłw na danej trasie,
- `reliability` - jako odsetek opĂłĹşnieĹ„ na danej trasie,
- `nb` - numerowane od 1, 2, 3 wedĹ‚ug kolumny `reliability`. W przypadku takich samych wartoĹ›ci powino zwrĂłciÄ‡ 1, 2, 2, 3... 
PamiÄ™taj o t

 W tym miejscu wykonaj każdą z kwerend, aby zainicjować strukturę bazy danych

In [102]:
print(separated[0])

/*
Tutaj zdefiniuj schemÄ™ `reporting`
*/

/*
Tutaj napisz definicjÄ™ widoku reporting.flight, ktĂłra:
- bÄ™dzie usuwaÄ‡ dane o lotach anulowanych `cancelled = 0`
- bÄ™dzie zawieraÄ‡ kolumnÄ™ `is_delayed`, zgodnie z wczeĹ›niejszÄ… definicjÄ… tj. `is_delayed = 1 if dep_delay_new > 0 else 0` (zaimplementowana w SQL)

WskazĂłwka:
- SQL - analiza danych > DzieĹ„ 4 Proceduralny SQL > WyraĹĽenia warunkowe
- SQL - analiza danych > Przygotowanie do zjazdu 2 > Widoki
*/
CREATE OR REPLACE VIEW reporting.flight as
SELECT 1



In [103]:
cursor.execute('CREATE SCHEMA IF NOT EXISTS reporting;')

In [104]:
cursor.execute(
    'CREATE OR REPLACE VIEW reporting.flight AS SELECT * , CASE WHEN dep_delay_new > 0 THEN 1 ELSE 0 END AS is_delayed FROM flight WHERE cancelled <> 0; '
)

In [105]:
print(separated[1])


/*
Tutaj napisz definicjÄ™ widoku reporting.top_reliability_roads, ktĂłra bÄ™dzie zawieraĹ‚a nastÄ™pujÄ…ce kolumny:
- `origin_airport_id`,
- `origin_airport_name`,
- `dest_airport_id`,
- `dest_airport_name`,
- `year`,
- `cnt` - jako liczba wykonananych lotĂłw na danej trasie,
- `reliability` - jako odsetek opĂłĹşnieĹ„ na danej trasie,
- `nb` - numerowane od 1, 2, 3 wedĹ‚ug kolumny `reliability`. W przypadku takich samych wartoĹ›ci powino zwrĂłciÄ‡ 1, 2, 2, 3... 
PamiÄ™taj o tym, ĹĽe w wyniku powinny pojawiÄ‡ siÄ™ takie trasy, na ktĂłrych odbyĹ‚o siÄ™ ponad 10000 lotĂłw.

WskazĂłwka:
- SQL - analiza danych > DzieĹ„ 2 Relacje oraz JOIN > JOIN
- SQL - analiza danych > DzieĹ„ 3 - Analiza danych > Grupowanie
- SQL - analiza danych > DzieĹ„ 1 Podstawy SQL > Aliasowanie
- SQL - analiza danych > DzieĹ„ 1 Podstawy SQL > Podzapytania
*/
CREATE OR REPLACE VIEW reporting.top_reliability_roads AS
SELECT 1



In [108]:
cursor.execute(
    "CREATE OR REPLACE VIEW reporting.top_reliability_roads AS SELECT a.origin_airport_id, b.name AS origin_airport_name, a.dest_airport_id, c.name AS dest_airport_name,a.year,b.name || ' - ' || c.name AS route,count(*) AS cnt, (COUNT(*) FILTER (WHERE dep_delay_new > 0) *1.0 / COUNT(*)) AS reliability, DENSE_RANK() OVER (reliability_window) as nb FROM flight AS a LEFT JOIN airport_list AS b USING (origin_airport_id) LEFT JOIN airport_list AS c ON a.dest_airport_id = c.origin_airport_id GROUP BY  a.origin_airport_id, b.name, a.dest_airport_id, c.name, a.year, b.name || ' - ' || c.name HAVING count(*) > 10000 WINDOW reliability_window AS (ORDER BY (COUNT(*) FILTER (WHERE dep_delay_new > 0) *1.0 / COUNT(*)) DESC) ORDER BY reliability DESC;"
)

In [106]:
print(separated[2])


/*
Tutaj napisz definicjÄ™ widoku reporting.year_to_year_comparision, ktĂłra bÄ™dzie zawieraĹ‚ nastÄ™pujÄ…ce kolumny:
- `year`
- `month`,
- `flights_amount`
- `reliability`
*/
CREATE OR REPLACE VIEW reporting.year_to_year_comparision AS
SELECT 1



In [109]:
cursor.execute(
    "CREATE OR REPLACE VIEW reporting.year_to_year_comparision AS SELECT year, month, count(*) as flight_amount, (COUNT(*) FILTER (WHERE dep_delay_new > 0) * 1.0 / COUNT(*)) AS reliability FROM flight GROUP BY year, month;"
)

In [110]:
print(separated[3])


/*
Tutaj napisz definicjÄ™ widoku reporting.day_to_day_comparision, ktĂłry bÄ™dzie zawieraĹ‚ nastÄ™pujÄ…ce kolumny:
- `year`
- `day_of_week`
- `flights_amount`
*/
CREATE OR REPLACE VIEW reporting.day_to_day_comparision AS
SELECT 1



In [111]:
cursor.execute(
    "CREATE OR REPLACE VIEW reporting.day_to_day_comparision AS SELECT year, day_of_week, count(*) as flight_amount FROM flight GROUP BY year, day_of_week;"
)

In [112]:
print(separated[4])


/*
Tutaj napisz definicjÄ™ widoku reporting.day_by_day_reliability, ktory bÄ™dzie zawieraĹ‚ nastÄ™pujÄ…ce kolumny:
- `date` jako zĹ‚oĹĽenie kolumn `year`, `month`, `day`, powinna byÄ‡ typu `date`
- `reliability` jako odsetek opĂłĹşnieĹ„ danego dnia

WskazĂłwki:
- formaty dat w postgresql: [klik](https://www.postgresql.org/docs/13/functions-formatting.html),
- jeĹ›li chcesz dodaÄ‡ zera na poczÄ…tek liczby np. `1` > `01`, posĹ‚uĹĽ siÄ™ metodÄ… `LPAD`: [przykĹ‚ad](https://stackoverflow.com/questions/26379446/padding-zeros-to-the-left-in-postgresql),
- do konwertowania ciÄ…gu znakĂłw na datÄ™ najwygodniej w Postgres uĹĽyÄ‡ `to_date`: [przykĹ‚ad](https://www.postgresqltutorial.com/postgresql-date-functions/postgresql-to_date/)
- do zĹ‚Ä…czenia kilku kolumn / wartoĹ›ci typu string, uĹĽywa siÄ™ operatora `||`, przykĹ‚adowo: SELECT 'a' || 'b' as example

Uwaga: Nie dodawaj tutaj na koĹ„cu srednika - przy uĹĽywaniu split, pojawi siÄ™ pusta kwerenda, co bÄ™dzie skutkowaĹ‚o pĂłĹşniejszym bĹ‚Ä™de

In [113]:
cursor.execute(
    "CREATE OR REPLACE VIEW reporting.day_by_day_reliability AS SELECT TO_DATE(year || '-' || LPAD(month::text, 2, '0') || '-' || LPAD(day_of_month::text, 2, '0'), 'YYYY-MM-DD') AS combined_date, (COUNT(*) FILTER (WHERE dep_delay_new > 0) * 1.0 / COUNT(*)) AS reliability FROM flight GROUP BY  year, month, day_of_month;"
)

 Zatwierdzenie wszystkich operacji na bazie, czyli stworzenie widoków

In [114]:
con.commit()

 ### Sprawdzenie

In [115]:
# Ten kod chyba wygląda znajomo....
# istnienie widoków możemy sprawdzić tak samo jak tabele
def check_if_table_exists(table_name):
    msg = f"Sprawdzam czy istnieje tabela {table_name}"
    print(msg)

    query = f"select 1 from {table_name}"
    # jeżeli tabela nie istnieje, ten krok zwróci wyjątek
    cursor.execute(query)
    print('OK!')

In [116]:
views_to_test = [
    'reporting.flight',
    'reporting.top_reliability_roads',
    'reporting.year_to_year_comparision',
    'reporting.day_to_day_comparision',
    'reporting.day_by_day_reliability'
]

In [117]:
for view in views_to_test:
    check_if_table_exists(view)

Sprawdzam czy istnieje tabela reporting.flight
OK!
Sprawdzam czy istnieje tabela reporting.top_reliability_roads
OK!
Sprawdzam czy istnieje tabela reporting.year_to_year_comparision
OK!
Sprawdzam czy istnieje tabela reporting.day_to_day_comparision
OK!
Sprawdzam czy istnieje tabela reporting.day_by_day_reliability
OK!


In [118]:
con.close()
msg = "Wszystko wygląda OK :) Możesz przejść do kolejnego zadania."
print(msg)

Wszystko wygląda OK :) Możesz przejść do kolejnego zadania.


 # Podsumownie
 W tym notatniku stworzyliśmy nowy schemat - `reporting`, którego zadaniem jest przygotowanie naszych danych
 do wizualizacji. Dalsza część pracy będzie polegała na wyświetleniu w wizualnie atrakcyjny sposób danych w interaktywnym raporcie stworzonym
 z pomocą `Dash`.